In [1]:

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import numpy as np

In [2]:
Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Select Data Release 2, default
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
Gaia.ROW_LIMIT=-1


## We first are going to introduce the DAML catalogue for clusters. The goal will be to create an algorithm which can find the cluster and then assign stars in the region to that cluster. A similar idea is employed in https://www.aanda.org/articles/aa/pdf/2002/27/aa2476.pdf

In [3]:
from astroquery.vizier import Vizier
#Vizier.ROW_LIMIT = -1

In [4]:
catalog_list=Vizier.find_catalogs('Dias+ 2002-2015')

#This is the DAML globular cluster catalogue 
#The warnings need to be dealt with

In [5]:
#An import of all their values
catalogs = Vizier.get_catalogs(catalog_list.values())

# Catalogs has load of different tables, the second one is the list of clusters

In [6]:
cluster_list=catalogs[1] #only has 50 rows

In [7]:
sorted_cluster_list=cluster_list[np.argsort(cluster_list['Dist'])]

sorted_cluster_list

P,Cluster,RAJ2000,DEJ2000,Class,Diam,Dist,E_B-V_,Age,pmRA,pmDE,Nc,K14,RV,o_RV,__Fe_H_,o__Fe_H_,TrType,WEBDA,Lynga
,,"""h:m:s""","""d:m:s""",,arcmin,pc,mag,[yr],mas / yr,mas / yr,,,km / s,,Sun,,,,
str1,str17,str8,str9,str2,float32,int16,float32,float32,float32,float32,int16,str3,float32,int16,float32,int16,str8,str5,str5
P,Blanco 1,00 04 07,-29 50 00,,70.0,269,0.010,7.796,20.17,3.00,27,K14,5.53,49,0.040,8,,WEBDA,Lynga
,Alessi 20,00 10 34,+58 45 36,e,40.0,600,0.220,6.480,8.73,-3.11,17,K14,-11.50,1,--,--,,WEBDA,Lynga
,NGC 7826,00 05 17,-20 41 30,r,20.0,620,0.030,9.340,6.48,-6.20,39,K14,-4.62,1,--,--,4-2-p-*,WEBDA,Lynga
P,NGC 133,00 31 19,+63 21 00,,7.0,630,0.600,7.000,-0.70,-0.40,8,K14,--,--,--,--,4-1-p-*,WEBDA,Lynga
P,NGC 225,00 43 39,+61 46 30,,12.0,657,0.274,8.114,-1.23,-0.06,89,K14,-28.00,1,--,--,,WEBDA,Lynga
,NGC 189,00 39 35,+61 05 42,,5.0,752,0.420,7.000,-0.36,-3.02,80,K14,--,--,--,--,,WEBDA,Lynga
,Berkeley 59,00 02 14,+67 25 00,,10.0,1000,1.241,6.100,-2.11,-1.20,2,K14,-5.17,3,--,--,3-3-m-n,WEBDA,Lynga
,King 1,00 22 04,+64 22 50,d,24.6,1080,0.760,9.600,-2.43,1.02,700,K14,-38.40,10,-0.010,10,2-2-r-*,WEBDA,Lynga


# There will be error on our measurements of the distance to our stars, reference Bailey Jones, so we want to order the clusters by the nearest ones
# We can look at the paper Dias for error on the distance measuremenet about the cluster, must be some error involved

The closest ones of course will have the smallest error when relating to Bailer Jones parallax inversion,
without doing anything rigarous we will take the closest star cluster and look at gaia data just by inverting the parallax to get
a measurement. We will take a window of twice the diameter, and depth twice the diameter.


In [8]:
#Taking the closest cluster
closest_cluster=sorted_cluster_list[0]
closest_cluster



P,Cluster,RAJ2000,DEJ2000,Class,Diam,Dist,E_B-V_,Age,pmRA,pmDE,Nc,K14,RV,o_RV,__Fe_H_,o__Fe_H_,TrType,WEBDA,Lynga
,,"""h:m:s""","""d:m:s""",,arcmin,pc,mag,[yr],mas / yr,mas / yr,,,km / s,,Sun,,,,
str1,str17,str8,str9,str2,float32,int16,float32,float32,float32,float32,int16,str3,float32,int16,float32,int16,str8,str5,str5
P,Blanco 1,00 04 07,-29 50 00,,70.0,269,0.010,7.796,20.17,3.00,27,K14,5.53,49,0.040,8,,WEBDA,Lynga


So we can see there is a diameter of 70.0 arcseconds, we will use a window of size 140.0 arcseconds.
We have a distance measurement of 269pc. We need to get distance estimates of the Gaia Data in that region

Next we are going to call in the GAIA data centered around the catalogued open cluster

In [48]:
#Looking at the cone around the point
right_as_center=closest_cluster['RAJ2000']
dec_center=closest_cluster['DEJ2000']
diam=closest_cluster['Diam']

#is the frame right
coord = SkyCoord(right_as_center+dec_center, unit=(u.hourangle, u.deg))
rad = u.Quantity(diam, u.arcminute)

r = Gaia.cone_search_async(coordinate=coord, radius=rad, verbose=True)
gaia_edr3=r.get_results()

Launched query: '
                SELECT
                  
                  *,
                  DISTANCE(
                    POINT('ICRS', ra, dec),
                    POINT('ICRS', 1.0291698321917175, -29.833330905250314)
                  ) AS dist
                FROM
                  gaiaedr3.gaia_source
                WHERE
                  1 = CONTAINS(
                    POINT('ICRS', ra, dec),
                    CIRCLE('ICRS', 1.0291698321917175, -29.833330905250314, 1.1666666666666667)
                  )
                ORDER BY
                  dist ASC
                '
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/async
Content-type = application/x-www-form-urlencoded
303 303
[('Date', 'Thu, 02 Dec 2021 18:46:16 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=blo

In [54]:
#Print the table
gaia_edr3


solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,phot_bp_rp_excess_factor,bp_rp,bp_g,g_rp,dr2_radial_velocity,dr2_radial_velocity_error,dr2_rv_nb_transits,dr2_rv_template_teff,dr2_rv_template_logg,dr2_rv_template_fe_h,l,b,ecl_lon,ecl_lat,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,deg,deg,deg,deg,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,,,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,deg,deg,deg,deg,
int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float64,float64,float64,float64,float64
1636042515805110273,Gaia EDR3 2320827909591325056,2320827909591325056,1592198863,2016.0,1.015981930167069,0.49987766,-29.824162726631137,0.41829798,-2.0865498538758396,0.5794813,-3.60072,0.44665593,-0.06717043567684688,0.60586464,0.44157633587533707,0.4926354,-0.10760638,0.36690092,0.50589794,-0.16185193,-0.3117634,-0.04303693,0.5220674,-0.07245915,-0.28661653,0.0006765611,280,0,278,2,0.088630415,292.29675,0.0,0.0,31,False,1.566035,--,--,--,--,--,--,--,33,18,0.887862,38,7,0,0.021938711,74.08484,0,0,1.0025731,0.28142568,0.20073365,0.10364383,0.7032929,-41.511887,-1.5073847,31.989674,14.577966,False,320,193.81962739049592,1.1130284,174.13718,19.968872,32,131.08735452588292,11.087876,11.822584,20.04464,31,121.73481299366311,6.5201693,18.6705,19.534359,0,0,0,0,0,1.3044199,0.5102806,0.07576752,0.4345131,--,--,0,--,--,--,15.632446089530552,-79.25182926532618,348.0416582704387,-27.543343853338374,0.01466101910210699
1636042515805110273,Gaia EDR3 2320827879527299328,2320827879527299328,491857494,2016.0,1.049900291744989,3.6796942,-29.825765274127917,4.589537,--,--,--,--,--,--,--,--,-0.73318756,--,--,--,-

Right so we have loads of error stuff here and there is going to be a lot of management of that.
Can we trust the distance estimates on this.

Either way we need to figure out the depth of this. How many we are going to accept.

Now we are going to do Bailer Jones data

In [55]:
Vizier.ROW_LIMIT = -1
bailer = Vizier.query_region(coord,
                                 radius=rad,
                                 catalog='I/352/gedr3dis')

In [56]:
bjones=bailer[0]
bjones


Source,RA_ICRS,DE_ICRS,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
,deg,deg,pc,pc,pc,pc,pc,pc,
int64,float64,float64,float64,float64,float64,float64,float64,float64,int32
2314546502741963904,0.96583661444,-30.99691720017,1894.10693000,1039.53943000,2835.88501000,3324.18384000,2762.83496000,3996.71777000,10033
2314546369597053696,0.99846703055,-30.98905473164,2219.46777000,1100.83350000,4682.46582000,857.79760700,715.87896700,1050.30249000,11033
2314546575755789056,0.95254011615,-30.98595207167,1306.44214000,771.99603300,2271.42285000,9738.49805000,8234.71582000,12241.02050000,20033
2314546708900403584,0.96710571964,-30.96621414806,1341.11975000,896.87835700,1917.54175000,1450.77966000,1067.83240000,1730.60181000,10033
2314546708900404352,0.97448703956,-30.96359882823,943.89025900,529.88793900,1755.20789000,15650.70410000,13054.46680000,19768.20700000,20033
2314546708900405888,0.96435353716,-30.95882424959,3412.70117000,2345.08203000,6707.69531000,6478.59277000,5281.82227000,8211.66797000,10033
2314548843498958976,0.80183494480,-30.97489751775,732.54840100,692.12103300,781.52258300,726.00061000,678.48059100,766.12420700,10033
2314548877858696960,0.78513614988,-30.97797343225,1382.96277000,1211.61682000,1550.66882000,1369.74341000,1241.45264000,1537.38184000,10033


solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,phot_bp_rp_excess_factor,bp_rp,bp_g,g_rp,dr2_radial_velocity,dr2_radial_velocity_error,dr2_rv_nb_transits,dr2_rv_template_teff,dr2_rv_template_logg,dr2_rv_template_fe_h,l,b,ecl_lon,ecl_lat,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,deg,deg,deg,deg,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,,,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,deg,deg,deg,deg,
int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float64,float64,float64,float64,float64
1636042515805110273,Gaia EDR3 2314847670143738880,2314847670143738880,1765952471,2016.0,359.92735106190923,0.92434955,-30.129170960063206,0.93685526,-7.990065420719655,1.1462334,-6.970714,3.4486806,2.897914417651553,1.7988225,1.8696228507203516,1.2223145,-0.030519625,0.13975944,-0.079917744,0.114613175,-0.24187557,0.029593157,0.47218543,0.18287687,-0.29743302,-0.26825836,180,0,174,6,14.677872,707.60565,6.53134,26.479067,95,False,--,0.68954015,0.25094962,-0.04638765,0.09766823,0.10127902,-0.12311738,-0.028333638,22,15,2.5644977,26,7,3,0.508651,54.49154,10,3,1.8971575,0.1905884,0.21351354,0.11873213,0.7764126,-59.343395,7.7959867,38.340195,15.451039,False,205,201.3154275279898,3.0599751,65.78989,19.927673,21,178.69368461619524,12.265472,14.568838,19.70827,21,171.9794036730463,9.431011,18.235521,19.159204,0,21,0,21,0,1.7419087,0.54906464,-0.21940422,0.76846886,--,--,0,--,--,--,15.075720898203544,-78.2666795512862,346.93751473216093,-27.393242772893014,0.999178517709517
1636042515805110273,Gaia EDR3 2320879968890542464,2320879968890542464,38848959,2016.0,0.8012788343889028,1.5727688,-29.7204671333642

In [62]:
#see how many matches we have
count=0
for i in range (0, len(gaia_edr3['source_id'])):

    if(gaia_edr3['source_id'][i] not in bjones['Source']):
        count+=1
count
count + len(bjones['Source'])-len(gaia_edr3['source_id'])

0

Okay this gives all of the data that has Bailer jones distance estimates

In [46]:
bailer[0].columns

<TableColumns names=('Source','RA_ICRS','DE_ICRS','rgeo','b_rgeo','B_rgeo','rpgeo','b_rpgeo','B_rpgeo','Flag')>

In [93]:
gaia_edr3=gaia_edr3[gaia_edr3['parallax']>=-1000] #there is probably a better way of getting rid of the zero values but we shouldnt have that

#Now im getting rid of the values with a nonzero parallax


In [95]:
len(gaia_edr3)==len(bjones)

#Great, so it works and now we have catalogues with the same values

True

# Now we need to add on the columns and merge them together, we order them by source code, that should maek it easier.



In [96]:
gaia_edr3=gaia_edr3[np.argsort(gaia_edr3['source_id'])]
bjones=bjones[np.argsort(bjones['Source'])]

In [120]:
False in (gaia_edr3['source_id']==bjones['Source'])

#these dataframes are weird but basically theyre the same
#moreover i think i shoud have a pipline of changing these to more readable stuff lol
 

False

In [128]:
np.hstack([gaia_edr3, bjones])  #fix

TypeError: invalid type promotion with structured datatype(s).